<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 8 [Assessment]:** RAG Evaluation</font>

<br>

Welcome to the last notebook of the course! In the previous notebook, you integrated a vector store solution into a RAG pipeline! In this notebook, you will take that same pipeline and evaluate it using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics!

<br>

### **Learning Objectives:**

- Learn how to integrate the techniques from prior notebooks to numerically approximate the goodness of your RAG pipeline.

- **Final Exercice**: ***By working through this notebook in the Course Environment,* you will be able to submit the coding component of the course!**

<br>

### **Questions To Think About:**

- As you go along, remember what our metrics actually represent. Should our pipeline pass these objectives? Is our judge LLM sufficient for evaluating the pipeline? Does a particular metric even matter for our use case?
- If we left the vectorstore-as-a-memory component in our chain, do you think it would still pass the evaluation? Additionally, is the evaluation useful for assessing vectorstore-as-a-memory performance? 

<br>

### **Environment Setup:**

In [19]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

----

<br>

## **Part 1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **Part 2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/comparison/pairwise_string/). 

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***





**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).

<br>

### **Task 1:** Pull In Your Document Retrieval Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [20]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 253 chunks

Sample Chunk:

Paper: Unified Tool Integration for LLMs: A Protocol-Agnostic Approach to Function Calling

Summary: The proliferation of tool-augmented Large Language Models (LLMs) has created
a fragmented ecosystem where developers must navigate multiple protocols,
manual schema definitions, and complex execution workflows. We address this
challenge by proposing a unified approach to tool integration that abstracts
protocol differences while optimizing execution performance. Our solution
demonstrates how protocol-agnostic design principles can significantly reduce
development overhead through automated schema generation, dual-mode concurrent
execution, and seamless multi-source tool management. Experimental results show
60-80% code reduction across integration scenarios, performance improvements up
to 3.1x through optimized concurrency, and full compatibility with existing
function calling standards. This work contributes both theoretical insights
into tool integration architecture and practical sol

<br>

### **Task 2: [Exercise]** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook! 

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN

# Context getter: extract input from dict, then retrieve docs and convert to string
context_getter = (
    itemgetter('input')  # Extract the input string from the dictionary
    | docstore.as_retriever() 
    | long_reorder 
    | docs2str
)

retrieval_chain = {'input': (lambda x: x)} | RunnableAssign({'context': context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
generator_chain = chat_prompt | llm
generator_chain = {'output': generator_chain} | RunnableLambda(output_puller)  ## GIVEN


## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

That's a great question! Did you know that we have evidence that fully autonomous AI agents can implement the core functionality of issues on several open-source repositories? This is a massive leap forward in AI development and has the potential to significantly speed up development in this setting.

According to the study in ["Measuring the Impact of Early-2025 AI on Experienced Open-Source Developer Productivity"], these AI agents were able to correctly implement the core functionality of issues, although they failed to fully satisfy all requirements. This is a big deal, and if progress continues, we might see significant speedups in open-source development.

It's also worth noting that the repositories used in this study are extremely large and mature, with an average of 1.1 million lines of code and 23,000 stars. Still, the AI agents were able to make meaningful progress on these complex projects. Pretty cool, huh?

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [22]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How do the AI tools used in the context of the study on experienced open-source developers ("Cursor Pro" 
and "Claude 3.5/3.7 Sonnet") compare to the type of AI modeled in the "Agentic-PPML" framework for large language 
model (LLM) inference?

Answer: The AI tools used in the study on experienced open-source developers are primarily used for code editing 
and natural language processing, whereas the "Agentic-PPML" framework models a type of AI that is designed 
specifically for intent understanding and delegating cryptographically secure inference to specialized models. This
implies that the AI tools used in the study are more focused on productivity and coding assistance, whereas the 
"Agentic-PPML" framework is more focused on ensuring data privacy and making privacy-preserving LLM inference 
practical.

QA Pair 2

Question: What are some potential factors that could contribute to the slowing down of experienced open-source 
developers' productivity when using AI tools, as discovered in a randomized controlled trial in the wild?

Answer: The authors of the study document find evidence that the following factors could contribute to the slowdown
effect: tasks defined before randomization (potentially leading to more verbose but functionally equivalent code), 
prior developer experience with AI tooling (only 44% of developers had prior experience using the Cursor IDE, while
93% had previously used LLMs), and project characteristics such as size and quality standards. Additionally, the 
authors suggest that experimental artifacts may not be the primary cause of the slowdown effect, given its 
robustness across various analyses. However, it's worth noting that these results may not imply that current AI 
systems are not useful in many realistic, economically relevant settings.

QA Pair 3

Question: How can AI agents be designed to evaluate the outputs of other AI models in a more reliable and 
cost-effective way, while also promoting trustworthy and scalable evaluation for next-generation language models?

Answer: A new paradigm, called "agent-as-a-judge," is emerging, which leverages AI agents' reasoning and 
perspective-taking abilities to assess the quality and safety of other models. This approach promises to provide a 
more scalable and nuanced alternative to human evaluation, and research has explored dynamic multi-agent debate 
frameworks, internal multi-agent debates, and judge module architectures that can continuously evaluate AI outputs 
in real-time.

<br>

### **Step 4:** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [23]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    # Extract just the question text (remove "Question: " prefix if present)
    question_text = q.replace("Question: ", "") if q.startswith("Question: ") else q
    
    # Use the RAG chain to get the answer
    rag_response = ""
    for token in rag_chain.stream(question_text):
        rag_response += token
    
    rag_answer = rag_response.strip()
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How do the AI tools used in the context of the study on experienced open-source developers ("Cursor Pro" 
and "Claude 3.5/3.7 Sonnet") compare to the type of AI modeled in the "Agentic-PPML" framework for large language 
model (LLM) inference?

RAG Answer: So, to answer your question, the AI tools used in the study on experienced open-source developers are 
primarily "Cursor Pro" and "Claude 3.5/3.7 Sonnet". On the other hand, the "Agentic-PPML" framework models a type 
of AI for large language model (LLM) inference.

According to the "Unified Tool Integration for LLMs: A Protocol-Agnostic Approach to Function Calling" document, 
the "Agentic-PPML" framework is an approach to enabling privileged and protected machine learning, which means it 
provides a secure way to integrate LLMs into various applications. This framework is designed to allow for more 
general-purpose LLMs, as opposed to the more specialized AI tools like Cursor Pro and Claude 3.5/3.7 Sonnet used in
the study.

The study primarily focuses on the impact of these specialized AI tools on developer productivity, whereas the 
"Agentic-PPML" framework is more focused on the underlying mechanics of integrating LLMs. Given that Cursor Pro and
Claude 3.5/3.7 Sonnet are designed to facilitate coding tasks, whereas the "Agentic-PPML" framework is a broader 
framework for integrating LLMs, the two are quite distinct. Therefore, it's challenging to make a direct comparison
between them. The study and the "Agentic-PPML" framework serve different purposes and cater to different needs.

QA Pair 2
Question: What are some potential factors that could contribute to the slowing down of experienced open-source 
developers' productivity when using AI tools, as discovered in a randomized controlled trial in the wild?

RAG Answer: You're referring to the study "Measuring the Impact of Early-2025 AI on Experienced Open-Source 
Developer Productivity". According to the results, several factors could be contributing to the slowdown in 
experienced open-source developers' productivity when using AI tools. 

Firstly, it's worth mentioning that the study found that **over-optimism about AI usefulness** played a role. 
Despite developers being familiar with the AI tools, they still overestimated the benefits, expecting a 24% 
reduction in implementation time, but actually experiencing a 19% increase in completion time.

Another factor that contributed to the slowdown was **high developer familiarity with repositories**. The study 
found that developers were working on large, popular repositories they were highly familiar with, which might have 
affected their productivity.

Additionally, the study proposes several AI-specific factors that could be contributing to the slowdown. These 
include:

*   **Fundamental reliability**: AI systems that are not reliable may slow down developers.
*   **Latency**: AI systems with high latency may also slow down developers.
*   **Elicitation**: AI systems that are not well-elicitated (e.g. via skilled prompting or scaffolding) may slow 
down developers.

These findings highlight a disconnect between perceived and actual AI impact on developer productivity. Expect 
experts in economics and machine learning to make different predictions about the effects of AI on productivity.

Finally, the study suggests that **further improvements to current AI systems** (e.g. better prompting, agent 
scaffolding, or domain-specific fine-tuning) could potentially yield positive speedup in this setting.

QA Pair 3
Question: How can AI agents be designed to evaluate the outputs of other AI models in a more reliable and 
cost-effective way, while also promoting trustworthy and scalable evaluation for next-generation language models?

RAG Answer: That's a great question! To design AI agents that can evaluate the outputs of other AI models more 
reliably and cost-effectively, researchers have proposed a few strategies.

First, we can build libraries of common evaluation dimensions (e.g., fluency, factuality) and modular persona 
templates that can be composed for a specific domain [1]. This can help AI agents understand what to look for when 
evaluating outputs. Additionally, multimodal AI models (which can handle images, audio, etc.) are emerging, and 
future agent-judges might need to evaluate outputs that aren't just text, such as image captions or generated 
graphs [1].

Another approach is to develop standard meta-evaluation benchmarks and datasets, which would allow for objective 
comparison of different evaluation strategies [1]. For example, a benchmark could include model outputs for various
tasks with careful human scores along multiple axes.

Researchers also suggest designing uncertainty estimates or triggers for fallback to human evaluation, which would 
make these systems safer in practice [1]. This means that AI agents could be designed to recognize when their own 
evaluation is uncertain or incomplete, and then fall back on human judgment to ensure accuracy.

Finally, exploring user experience and dynamic evaluation could improve the performance of AI agents. For instance,
an AI tutoring agent might assess its own answer before presenting it to a student, in real-time. This could be 
done by training a smaller internal critic model that runs alongside the main model, allowing the AI agent to 
evaluate its own outputs on the fly [1]. 

These approaches aim to make AI-based evaluation more reliable, cost-effective, and trustworthy for next-generation
language models.

References:

[1] Yu, F. (2025). When AIs Judge AIs: The Rise of Agent-as-a-Judge Evaluation for LLMs.

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [27]:
## TODO: Adapt this prompt for whichever LLM you're actually interested in using. 
## If it's llama, maybe system message would be good?
eval_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an expert evaluator. You must respond with EXACTLY [1] or [2], nothing else in the brackets."
     " Be fair but recognize that detailed, well-sourced answers are usually better."
    ),
    ("user", 
     "Compare these two answers. Assume Answer 1 is acceptable baseline.\n\n"
     "SCORING:\n"
     "[1] Answer 2 is worse than Answer 1\n" 
     "[2] Answer 2 is better than Answer 1\n\n"
     "IMPORTANT: Start your response with [1] or [2] in brackets.\n\n"
     "{qa_trio}\n\n"
     "EVALUATION:"
    )
])

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How do the AI tools used in the context of the study on experienced open-source developers 
("Cursor Pro" and "Claude 3.5/3.7 Sonnet") compare to the type of AI modeled in the "Agentic-PPML" framework for 
large language model (LLM) inference?

Synth Answer: Answer: The AI tools used in the study on experienced open-source developers are primarily used for 
code editing and natural language processing, whereas the "Agentic-PPML" framework models a type of AI that is 
designed specifically for intent understanding and delegating cryptographically secure inference to specialized 
models. This implies that the AI tools used in the study are more focused on productivity and coding assistance, 
whereas the "Agentic-PPML" framework is more focused on ensuring data privacy and making privacy-preserving LLM 
inference practical.

RAG Answer: So, to answer your question, the AI tools used in the study on experienced open-source developers are 
primarily "Cursor Pro" and "Claude 3.5/3.7 Sonnet". On the other hand, the "Agentic-PPML" framework models a type 
of AI for large language model (LLM) inference.

According to the "Unified Tool Integration for LLMs: A Protocol-Agnostic Approach to Function Calling" document, 
the "Agentic-PPML" framework is an approach to enabling privileged and protected machine learning, which means it 
provides a secure way to integrate LLMs into various applications. This framework is designed to allow for more 
general-purpose LLMs, as opposed to the more specialized AI tools like Cursor Pro and Claude 3.5/3.7 Sonnet used in
the study.

The study primarily focuses on the impact of these specialized AI tools on developer productivity, whereas the 
"Agentic-PPML" framework is more focused on the underlying mechanics of integrating LLMs. Given that Cursor Pro and
Claude 3.5/3.7 Sonnet are designed to facilitate coding tasks, whereas the "Agentic-PPML" framework is a broader 
framework for integrating LLMs, the two are quite distinct. Therefore, it's challenging to make a direct comparison
between them. The study and the "Agentic-PPML" framework serve different purposes and cater to different needs.

Synth Evaluation: [2]

Set 2

Question: Question: What are some potential factors that could contribute to the slowing down of experienced 
open-source developers' productivity when using AI tools, as discovered in a randomized controlled trial in the 
wild?

...

Synth Evaluation: [2]

Set 3

Question: Question: How can AI agents be designed to evaluate the outputs of other AI models in a more reliable and
cost-effective way, while also promoting trustworthy and scalable evaluation for next-generation language models?

Synth Answer: Answer: A new paradigm, called "agent-as-a-judge," is emerging, which leverages AI agents' reasoning 
and perspective-taking abilities to assess the quality and safety of other models. This approach promises to 
provide a more scalable and nuanced alternative to human evaluation, and research has explored dynamic multi-agent 
debate frameworks, internal multi-agent debates, and judge module architectures that can continuously evaluate AI 
outputs in real-time.

RAG Answer: That's a great question! To design AI agents that can evaluate the outputs of other AI models more 
reliably and cost-effectively, researchers have proposed a few strategies.

First, we can build libraries of common evaluation dimensions (e.g., fluency, factuality) and modular persona 
templates that can be composed for a specific domain [1]. This can help AI agents understand what to look for when 
evaluating outputs. Additionally, multimodal AI models (which can handle images, audio, etc.) are emerging, and 
future agent-judges might need to evaluate outputs that aren't just text, such as image captions or generated 
graphs [1].

Another approach is to develop standard meta-evaluation benchmarks and datasets, which would allow for objective 
comparison of different evaluation strategies [1]. For example, a benchmark could include model outputs for various
tasks with careful human scores along multiple axes.

Researchers also suggest designing uncertainty estimates or triggers for fallback to human evaluation, which would 
make these systems safer in practice [1]. This means that AI agents could be designed to recognize when their own 
evaluation is uncertain or incomplete, and then fall back on human judgment to ensure accuracy.

Finally, exploring user experience and dynamic evaluation could improve the performance of AI agents. For instance,
an AI tutoring agent might assess its own answer before presenting it to a student, in real-time. This could be 
done by training a smaller internal critic model that runs alongside the main model, allowing the AI agent to 
evaluate its own outputs on the fly [1]. 

These approaches aim to make AI-based evaluation more reliable, cost-effective, and trustworthy for next-generation
language models.

References:

[1] Yu, F. (2025). When AIs Judge AIs: The Rise of Agent-as-a-Judge Evaluation for LLMs.

Synth Evaluation: [2]

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [28]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 1.0


----

<br>

## **Part 4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action. 

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/stable/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine. 

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/v0.1/docs/modules/agents/) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.

----

<br>

## **Part 5: [Assessment]** Evaluating For Credit

Welcome to the last exercise of the course! Hopefully you've enjoyed the material and are ready to actually get credit for these notebooks! For this part:

- **Make sure you're in the course environment**
- **Make sure `docstore_index/` has been uploaded to the course environment...**
    - **...and contains [at least one Arxiv paper](https://arxiv.org/search/advanced) which has been updated recently.**
- **Make sure you don't have some old session of [`09_langserve.ipynb`](09_langserve.ipynb) already occupying the port. Your assessment requires you to implement the new `/retriever` and `/generator` endpoints!!**

**Objective:** On launch, [**`frontend/frontend_block.py`**](frontend/frontend_block.py) had several lines of code which trigger the course pass condition. Your objective is to invoke that series of commands by using your pipeline to pass the **Evaluation** check! Recall [`09_langserve.ipynb`](09_langserve.ipynb) and use it as a starting example! As a recommendation, consider duplicating it so that you can keep the original as an authoritative reference. 

**Once Finished:** While your course environment is still open, please navigate back to your course environment launcher area and click the **"Assess Task"** button! After that, you're all done!

In [26]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

<IPython.core.display.Javascript object>

----

<br>

## <font color="#76b900">**Congratulations On Completing The Course**</font>

Hopefully this course was not only exciting and challenging, but also adequately prepared you for work on the cutting edge of LLM and RAG system development! Going forward, you should have the skills necessary to tackle industry-level challenges and explore RAG deployment with open-source models and frameworks.

**Some NVIDIA-specific releases related to this that you may find interesting include:**
- [**NVIDIA NIM**](https://www.nvidia.com/en-us/ai/), which offers microservice spinup routines that can be deployed on local compute.
- [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) is the current recommended framework for deploying GPU-accelerated LLM model engines in production settings.
- [**NVIDIA's Generative AI Examples Repo**](https://github.com/NVIDIA/GenerativeAIExamples), which includes the current canonical microservice example application and will be updated with new resources as new production workflows get released.
- [**The Knowledge-Based Chatbot Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief) which discusses additional publicly-accessible details on productionalizing RAG systems.

**Additionally, some key topics you may be interested in delving more into include:**
- [**LlamaIndex**](https://www.llamaindex.ai/), which has strong components that can augment and occasionally improve upon the LangChain RAG features.
- [**LangSmith**](https://docs.smith.langchain.com/), an upcoming agent productionalization service offered by LangChain.
- [**Gradio**](https://www.gradio.app/), though touched on in the course, has many more interface options which will be worth investigating. For inspiration, consider checking out [**HuggingFace Spaces**](https://huggingface.co/spaces) for examples.
- [**LangGraph**](https://python.langchain.com/docs/langgraph/) is a framework for graph-based LLM orchestration, and is a natural next step forward for those interested in [multi-agent workflows](https://blog.langchain.dev/langgraph-multi-agent-workflows/).
- [**DSPy**](https://github.com/stanfordnlp/dspy), a flow engineering framework that allows you to optimize LLM orchestration pipelines based on empirical performance results.

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>